# Get List

## Get Contents

## Linked In 접속

In [1]:
from scrapers import LinkedIn, DGLogger
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json
with open("./identity.json", "r") as f:
    identity = json.load(f)
linkedin = LinkedIn()
driver = linkedin.init_selenium()

## 채용공고 데이터 가져오기
<img style="width:85%;height:350px" src="./images/linkedin_index.png" />

**링크드인 채용공고 페이지는 한 페이지에서, 채용공고 리스트와 채용공고 내용을 한번에 보여줌**
- GET 방식의 파라미터 값을 채용공고의 Unique한 ID를 받음
- 채용공고 리스트를 보여주기 위한 다른 조건을 파라미터 값으로 받음
  - JobID가 주어지지 않을 경우, 조건값에 따른 공고 리스트를 임의의 규칙에 따라 정렬된 리스트 중 하나의 값을 받음
  - 규칙은 JobID에 따른 것이 아님
  - 기본적으로 로그인한 회원의 정보에 따라서 파라미터 값을 받음
    - ex) 지역코드를 입력하지 않으면, 회원이 있는 지역의 공고를 보여줌

**조건 값에 종류가 채용공고에 나타나 있지 않음**
- 조건 값에 따른 채용공고 정보를 알기 위해서는 리스트를 출력하여야 한다.
- 모든 데이터를 수집하기 위해서는 각각의 파라미터 값을 출력하는 것이 필요
  

### 채용 공고 추출 데이터(Sign in)
target_link = "https://www.linkedin.com/jobs/view/{job_id}/"  

In [4]:
# identity
_email = identity["_email"]
_pw = identity["_pw"]
linkedin.login_linkedin(f"{_email}",f"{_pw}")

False

In [3]:
import re
from datetime import datetime, timedelta

job_id = 3980762432
contents["job_id"] = job_id
target_link= f"https://www.linkedin.com/jobs/view/{job_id}/"
driver.get(target_link)
# Title
_list = driver.find_elements(By.CSS_SELECTOR,"h1")

# error 
if len(_list) != 1:
    print("error")
contents = {}

contents["title"] = _list[0].text    
#overview
overview = driver.find_element(By.CSS_SELECTOR,".mt2.mb2").text.split('\n')
contents['wr'] = overview[0]
contents['etc']= ";".join(overview[1:])

#tech stack
#modal up
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver.find_elements(By.CSS_SELECTOR, "button.mv5.t-16")[0].click()
WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CSS_SELECTOR,".job-details-skill-match-status-list__unmatched-skill")))
_list = driver.find_elements(By.CSS_SELECTOR,".job-details-skill-match-status-list__unmatched-skill")
tech_stacks = []
for i in _list:
    _str = i.text 
    tech_stacks.append(_str[:_str.find("\n")])
#modal down
driver.find_elements(By.CSS_SELECTOR,".artdeco-button.artdeco-button--circle.artdeco-button--muted.artdeco-button--2.artdeco-button--tertiary.ember-view.artdeco-modal__dismiss")[0].click()
tech_stacks = ";".join(tech_stacks)
contents['tech_stacks'] = tech_stacks

WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CSS_SELECTOR,".job-details-jobs-unified-top-card__primary-description-container")))
contents["geo"],_,apt = driver.find_element(By.CSS_SELECTOR,".job-details-jobs-unified-top-card__primary-description-container").text.split("·")
contents["apt"]= int(re.compile("[0-9]+").findall(apt)[0])
driver.find_element(By.CSS_SELECTOR,".jobs-description__footer-button.t-14.t-black--light.t-bold.artdeco-card__action.artdeco-button.artdeco-button--icon-right.artdeco-button--3.artdeco-button--fluid.artdeco-button--tertiary.ember-view").click()
WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CSS_SELECTOR,".jobs-box--fadein.jobs-box--full-width.jobs-box--with-cta-large.jobs-description.artdeco-card.jobs-poster--is-expanded-redesign.mt4")))
contents['content'] = driver.find_element(By.CSS_SELECTOR,"#job-details > div.mt4").text
driver.find_element(By.CSS_SELECTOR,".jobs-description__footer-button.t-14.t-black--light.t-bold.artdeco-card__action.artdeco-button.artdeco-button--icon-right.artdeco-button--3.artdeco-button--fluid.artdeco-button--tertiary.ember-view").click()
WebDriverWait(driver,3).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,".jobs-box--fadein.jobs-box--full-width.jobs-box--with-cta-large.jobs-description.jobs-description--is-truncated.jobs-description--is-truncated-poster.artdeco-card.mt4")))
contents['get_date'] = datetime.today().strftime("%Y-%m-%d_%H:%M:%s")
#company
contents['company']= driver.find_element(By.CSS_SELECTOR,".ember-view.link-without-visited-state.inline-block.t-black").text

In [4]:
contents

{'title': 'English Teacher',
 'wr': 'On-site Full-time',
 'etc': 'Skills: English Teaching, Teaching English as a Foreign Language, +5 more;Applicant review time is typically 4 days;Learn more;See how you compare to over 100 other applicants. Try Premium for ₩0',
 'tech_stacks': 'Business English;English Teaching;English as a Second Language (ESL);Language Teaching;Teaching;Teaching English as a Foreign Language;Teaching English as a Second Language',
 'geo': 'Seoul, South Korea ',
 'apt': 100,
 'content': "We are seeking a highly motivated and dedicated English Language Teacher who is eager to embark on an exciting career abroad. The ideal candidate will possess a genuine passion for teaching English and the ability to inspire students to reach their full potential.\n\nIf you are passionate about teaching English and eager to embark on a new and exciting career abroad, we encourage you to apply for this position.\n\nWe offer competitive compensation packages, professional development 

## 추출 데이터 형식(로그인 X)

1. 채용공고 제목
2. 회사
3. 내용
4. 전문성
5. 근무형태
6. 직무 분야
7. 산업

In [5]:
from crawl_code.linkedin.scrapers import logout_linkedin

logout_linkedin(driver)

In [6]:
job_id = 3980762432
target_link= f"https://www.linkedin.com/jobs/view/{job_id}/"
driver.get(target_link)
_list = driver.find_elements(By.CSS_SELECTOR,"li.description__job-criteria-item")
contents["jf"] = _list[2].text[_list[2].text.find("\n")+1:]
contents["industry"] = _list[3].text[_list[3].text.find("\n")+1:]

In [10]:
contents

{'title': 'English Teacher',
 'wr': 'On-site Full-time',
 'etc': 'Skills: English Teaching, Teaching English as a Foreign Language, +5 more;Applicant review time is typically 4 days;Learn more;See how you compare to over 100 other applicants. Try Premium for ₩0',
 'tech_stacks': 'Business English;English Teaching;English as a Second Language (ESL);Language Teaching;Teaching;Teaching English as a Foreign Language;Teaching English as a Second Language',
 'geo': 'Seoul, South Korea ',
 'apt': 100,
 'content': "We are seeking a highly motivated and dedicated English Language Teacher who is eager to embark on an exciting career abroad. The ideal candidate will possess a genuine passion for teaching English and the ability to inspire students to reach their full potential.\n\nIf you are passionate about teaching English and eager to embark on a new and exciting career abroad, we encourage you to apply for this position.\n\nWe offer competitive compensation packages, professional development 

In [42]:
import requests
from bs4 import BeautifulSoup
data = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"}
soup = BeautifulSoup(requests.get("https://www.linkedin.com/jobs/view/3990451742/",data=data).text)

In [56]:
import json
_dict = json.loads(soup.find('script',attrs={"type":"application/ld+json"}).text)
expr=[]
if "educationRequirements" in  _dict:
    expr.append(_dict["educationRequirements"]['credentialCategory'])
if "experienceRequirements" in _dict:
    expr.append(str(_dict["experienceRequirements"]['monthsOfExperience']) + " month")
contents["reqr"] = ";".join(expr)
contents['dateposted']= _dict['datePosted']

In [58]:
contents

{'title': 'English Teacher',
 'wr': 'On-site Full-time',
 'etc': 'Skills: English Teaching, Teaching English as a Foreign Language, +5 more;Applicant review time is typically 4 days;Learn more;See how you compare to over 100 other applicants. Try Premium for ₩0',
 'tech_stacks': 'Business English;English Teaching;English as a Second Language (ESL);Language Teaching;Teaching;Teaching English as a Foreign Language;Teaching English as a Second Language',
 'geo': 'Seoul, South Korea ',
 'apt': 100,
 'content': "We are seeking a highly motivated and dedicated English Language Teacher who is eager to embark on an exciting career abroad. The ideal candidate will possess a genuine passion for teaching English and the ability to inspire students to reach their full potential.\n\nIf you are passionate about teaching English and eager to embark on a new and exciting career abroad, we encourage you to apply for this position.\n\nWe offer competitive compensation packages, professional development 